<a href="https://colab.research.google.com/github/chinmay002/NLP/blob/main/Chapter_4_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip install transformers

In [97]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np


In [98]:
!pip install datasets

In [99]:
from datasets import get_dataset_config_names,load_dataset

xtreme_subsets = get_dataset_config_names('xtreme')


In [100]:
panx_subsets = [s for s in xtreme_subsets if s.startswith('PAN')]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [101]:
load_dataset('xtreme',name = 'PAN-X.de')

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [102]:
from collections import defaultdict
from datasets import DatasetDict

#actua; proportion of population speaking lanhuages in Swiss
langs = ['de','fr','it','en']
fracs = [0.629,0.229,0.084,0.059]

panx_ch = defaultdict(DatasetDict)

for lang,frac in zip(langs,fracs):
  #load data
  ds = load_dataset('xtreme',name = f'PAN-X.{lang}')
  #select only assigned prop data
  for split in ds:
    panx_ch[lang][split] = (ds[split].shuffle(seed = 0).select(range(int(frac * ds[split].num_rows))))

In [103]:
ds = load_dataset('xtreme',name='PAN-X.de')
ds['train'].select(range(int(ds['train'].num_rows*0.629)))

Dataset({
    features: ['tokens', 'ner_tags', 'langs'],
    num_rows: 12580
})

In [104]:
panx_ch['de']

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 12580
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 6290
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 6290
    })
})

In [105]:
panx_ch['de']['train'][0]
#ner_tags mapping each entity to a class ID

{'tokens': ['2.000',
  'Einwohnern',
  'an',
  'der',
  'Danziger',
  'Bucht',
  'in',
  'der',
  'polnischen',
  'Woiwodschaft',
  'Pommern',
  '.'],
 'ner_tags': [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0],
 'langs': ['de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de']}

In [141]:
tags = panx_ch['de']['train'].features['ner_tags'].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [107]:
def create_tag_names(batch):

  return {'new_tag_names':[tags.int2str(idx) for idx in  batch['ner_tags']]}

panx_de = panx_ch['de'].map(create_tag_names)

In [108]:
from collections import Counter
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
  for row in dataset["new_tag_names"]:
    for tag in row:
      if tag.startswith("B"):
        tag_type = tag.split("-")[1]
        split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


##SentencePieceToenizer

In [109]:
from huggingface_hub import notebook_login
notebook_login()

In [110]:
from transformers import AutoTokenizer

bert_model_name = 'bert-base-cased'
xlmr_model_name = 'xlm-roberta-base'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)


In [111]:
text = 'Chinmay Maganur from Haveri Karnataka'
print(bert_tokenizer(text).tokens())
print(xlmr_tokenizer(text).tokens())

['[CLS]', 'Chin', '##ma', '##y', 'Ma', '##gan', '##ur', 'from', 'Have', '##ri', 'Karnataka', '[SEP]']
['<s>', '▁Chin', 'may', '▁Maga', 'nur', '▁from', '▁Have', 'ri', '▁Karnataka', '</s>']


#Custom Model for own task

In [112]:
#custome classifcation head for XLM-R.  XLM uses Roberta , we will use ROBERTA as base model. and augument with XLM-R settings

In [113]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel,RobertaPreTrainedModel

In [114]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load a pre-trained token-level classification model and tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Define the input text
text = "She walks to the park."

# Tokenize the text
input_ids = tokenizer.encode(text, return_tensors='pt')
print(input_ids)
# Perform token-level classification
with torch.no_grad():
    output = model(input_ids)

print(output)

# Get the predicted logits for each token
logits = output.logits
print(logits)
# Predicted labels can be obtained by finding the argmax of the logits
predicted_labels = logits.argmax(dim=2)[0].tolist()

# Map labels to their respective tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
token_label_pairs = list(zip(tokens, predicted_labels))

# Display the results
for token, label_id in token_label_pairs:
    label = model.config.id2label[label_id]
    print(f"{token}: {label}")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 1153, 7104, 1106, 1103, 2493,  119,  102]])
TokenClassifierOutput(loss=None, logits=tensor([[[ 9.7553, -1.8620, -1.1356, -1.7457, -1.8517, -1.6100, -0.9773,
          -1.6794,  0.7354],
         [ 9.8082, -2.3094, -0.6437, -2.7310, -0.5595, -2.2548, -0.5026,
          -2.4405, -0.1142],
         [10.8961, -2.0296, -0.6506, -2.4348, -1.2476, -1.8974, -0.8352,
          -2.2057, -0.0515],
         [10.8708, -1.8079, -0.5061, -2.3962, -1.3077, -1.9588, -0.6790,
          -1.6600,  0.2246],
         [11.0932, -2.2698, -1.0893, -2.4386, -1.5270, -1.9758, -0.5125,
          -1.8652,  0.3215],
         [ 8.1143, -2.3099, -0.3689, -2.3250, -1.4927, -1.9463,  0.6469,
          -2.0029,  3.1118],
         [ 9.7552, -1.8620, -1.1356, -1.7457, -1.8517, -1.6100, -0.9773,
          -1.6794,  0.7354],
         [ 9.7553, -1.8620, -1.1356, -1.7457, -1.8517, -1.6100, -0.9773,
          -1.6794,  0.7354]]]), hidden_states=None, attentions=None)
tensor([[[ 9.7553, -1.8620, -1.1356, -1.7457,

In [115]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig

  def __init__(self,config):
    super().__init__(config)
    self.num_labels = config.num_labels
    self.roberta = RobertaModel(config,add_pooling_layer = False)#pool=false to ensure all tokens hiddenstates are returned and not just CLS
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size,config.num_labels)
    self.init_weights() #inherit from RobertaPreTrained  to load the pretreined weights for model body and randomly initialize the wieghts of our tokens classif head


  def forward(self,input_ids=None,attention_mask=None,token_type_ids =None,labels=None,**kwargs):
    outputs = self.roberta(input_ids,attention_mask = attention_mask,token_type_ids = token_type_ids,**kwargs)
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1,self.num_labels),labels.view(-1))
    return TokenClassifierOutput(loss= loss,logits = logits, hidden_states=outputs.hidden_states,attentions = outputs.attentions)

In [116]:
logits.view(-1)

tensor([ 9.7553, -1.8620, -1.1356, -1.7457, -1.8517, -1.6100, -0.9773, -1.6794,
         0.7354,  9.8082, -2.3094, -0.6437, -2.7310, -0.5595, -2.2548, -0.5026,
        -2.4405, -0.1142, 10.8961, -2.0296, -0.6506, -2.4348, -1.2476, -1.8974,
        -0.8352, -2.2057, -0.0515, 10.8708, -1.8079, -0.5061, -2.3962, -1.3077,
        -1.9588, -0.6790, -1.6600,  0.2246, 11.0932, -2.2698, -1.0893, -2.4386,
        -1.5270, -1.9758, -0.5125, -1.8652,  0.3215,  8.1143, -2.3099, -0.3689,
        -2.3250, -1.4927, -1.9463,  0.6469, -2.0029,  3.1118,  9.7552, -1.8620,
        -1.1356, -1.7457, -1.8517, -1.6100, -0.9773, -1.6794,  0.7354,  9.7553,
        -1.8620, -1.1356, -1.7457, -1.8517, -1.6100, -0.9773, -1.6794,  0.7354])

In [117]:
index2tag = {idx:tag for idx,tag in enumerate(tags.names)}
tag2index = {tag:idx for idx,tag in enumerate(tags.names)}

In [118]:
print(index2tag)
print(tag2index)


{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [119]:
from transformers import AutoConfig
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,num_labels = tags.num_classes,id2label = index2tag,label2id = tag2index)
#when we load model with automodel.from_pretrained. config file is automatically downloaded , if we want to modify something like num_classes ,
#then we can load the config first with params we would like to customize

In [120]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name,config = xlmr_config)).to(device)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [121]:
text ='Jack Sparrow loves New York!'
xlmr_tokens = xlmr_tokenizer(text).tokens()
input_ids = xlmr_tokenizer.encode(text,return_tensors ='pt')
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


In [122]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs,dim = -1)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequence: 10
Shape of outputs: torch.Size([1, 10, 7])


In [123]:
print(torch.argmax(outputs,dim=-1))
print(predictions)
print(outputs)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([[[-6.4615e-02,  7.4301e-01, -2.1983e-02,  5.0029e-02, -4.5834e-01,
          -1.6941e-01, -3.6184e-01],
         [ 1.3455e-01,  4.2784e-01,  3.4440e-02,  7.0764e-02, -4.6243e-01,
           8.8729e-03, -2.4587e-01],
         [ 9.2424e-02,  6.1125e-01,  8.5095e-02,  1.5196e-01, -4.8387e-01,
          -1.1778e-05, -2.7675e-01],
         [ 1.1490e-01,  5.4942e-01,  1.6539e-01,  4.8492e-02, -4.1260e-01,
          -1.9818e-02, -2.4665e-01],
         [ 4.9413e-02,  4.7403e-01,  3.7864e-02, -3.9754e-03, -4.6743e-01,
          -1.1634e-02, -3.1619e-01],
         [ 7.5898e-02,  5.9612e-01,  2.7333e-02, -4.3995e-02, -3.9091e-01,
          -4.5050e-02, -3.1514e-01],
         [ 6.2879e-02,  4.9621e-01, -3.6217e-02, -1.5974e-02, -3.9420e-01,
           6.0270e-03, -3.1164e-01],
         [ 7.7874e-02,  4.4654e-01,  5.8096e-02,  3.3276e-02, -4.0231e-01,
          -5.4143e-02, -3

In [124]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens,preds])

,0,1,2,3,4,5,6,7,8,9
0,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
1,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER


In [125]:
def tag_text(text,tags,model,tokenizer):
  tokens = tokenizer(text).tokens()
  input_ids = xlmr_tokenizer.encode(text,return_tensors ='pt')
  outputs = model(input_ids.to(device)).logits
  predictions = torch.argmax(outputs,dim = -1)
  preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
  return pd.DataFrame([xlmr_tokens,preds])

In [126]:
xlmr_tokenizer

XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [127]:
words ,labels = panx_de['train'][0]['tokens'],panx_de['train'][0]['ner_tags']
words,labels

(['2.000',
  'Einwohnern',
  'an',
  'der',
  'Danziger',
  'Bucht',
  'in',
  'der',
  'polnischen',
  'Woiwodschaft',
  'Pommern',
  '.'],
 [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0])

In [128]:
tokenized_input = xlmr_tokenizer(panx_de['train'][0]['tokens'],is_split_into_words=True)#we cant send tokens to encode, we need to send text
xlmr_tokenizer(panx_de['train'][0]['tokens'],is_split_into_words=True)

{'input_ids': [0, 70101, 176581, 19, 142, 122, 2290, 708, 1505, 18363, 18, 23, 122, 127474, 15439, 13787, 14, 15263, 18917, 663, 6947, 19, 6, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [129]:
xlmr_tokenizer(panx_de['train'][0]['tokens'])

{'input_ids': [[0, 70101, 2], [0, 176581, 19, 2], [0, 142, 2], [0, 122, 2], [0, 2290, 708, 1505, 2], [0, 18363, 18, 2], [0, 23, 2], [0, 122, 2], [0, 127474, 15439, 2], [0, 13787, 14, 15263, 18917, 2], [0, 663, 6947, 19, 2], [0, 6, 5, 2]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [130]:
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
pd.DataFrame([tokens],index = ['Tokens'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


In [131]:
#'Einwohnern' is diveide into two words. We need a way to mask the sub represntation after first words. tokenized_input provides word_ids()
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens,word_ids],index = ['tokens','words_ids'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
words_ids,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


In [132]:
prev_word_idx = None
label_ids = []
for word_idx in word_ids :
  if word_idx is None or word_idx==prev_word_idx:
    label_ids.append(-100)
  elif word_idx != prev_word_idx:
    label_ids.append(labels[word_idx])
  prev_word_idx = word_idx

labels = [index2tag[l] if l!= -100 else "IGN"for  l in label_ids]
index = ['Tokens','WOrd IDs','label ids','labels']

In [133]:
pd.DataFrame([tokens,word_ids,label_ids,labels],index=index)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
WOrd IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None
label ids,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
labels,IGN,O,O,IGN,O,O,B-LOC,IGN,IGN,I-LOC,...,B-LOC,IGN,IGN,IGN,I-LOC,IGN,IGN,O,IGN,IGN


In [134]:
#why -100, pytorch cross entropy loss has an attributte called ignore_index whose value is -100, index is ignore dureing training, so these tokens are ignores

In [135]:
#for a complete data
def tokenize_and_align_labels(examples):
  tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True,
  is_split_into_words=True)
  labels = []
  for idx, label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=idx)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
      else:
        label_ids.append(label[word_idx])
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

def encode_panx_dataset(corpus):
  return corpus.map(tokenize_and_align_labels,batched=True,remove_columns = ['langs','ner_tags','tokens'])

In [136]:
panx_de_encoded = encode_panx_dataset(panx_ch['de'])

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

In [137]:
panx_de_encoded['train'][0]

{'input_ids': [0,
  70101,
  176581,
  19,
  142,
  122,
  2290,
  708,
  1505,
  18363,
  18,
  23,
  122,
  127474,
  15439,
  13787,
  14,
  15263,
  18917,
  663,
  6947,
  19,
  6,
  5,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  -100,
  0,
  0,
  5,
  -100,
  -100,
  6,
  -100,
  0,
  0,
  5,
  -100,
  5,
  -100,
  -100,
  -100,
  6,
  -100,
  -100,
  0,
  -100,
  -100]}

In [138]:
import numpy as np
def align_predictions(predictions,label_ids):
  preds = np.argmax(predictions,axis=2)
  batch_size,seq_len = preds.shape
  labels_list,preds_list = [],[]
  for batch_idx in range(batch_size):
    example_labels,example_preds = [],[]
    for seq_idx in range(seq_len):
      if label_ids[batch_idx,seq_idx]!=-100:
        example_labels.append(index2tag[label_ids[batch_idx,seq_idx]])
        example_preds.append(index2tag[preds[batch_idx,seq_idx]])
  labels_list.append(example_labels)
  preds_list.append(example_preds)

  return labels_list,preds_list

In [139]:
!pip install transformers[torch]

In [140]:
from transformers import TrainingArguments
num_epochs = 3
batch_size = 24
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-de"
training_args = TrainingArguments(output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
                                  save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
                                  logging_steps=logging_steps,)

ImportError: ignored

In [ ]:
from seqeval.metrics import f1_score
def compute_metrics(eval_pred):
  y_pred,y_true = align_predictions(eval_pred.predictions,eval_pred.label_ids)
  return {'f1':f1_score(y_true,y_pred)}

In [ ]:
#padding each input sequence length to a largest sequence length in a batch
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)


In [ ]:
def model_init():
  return (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name,config =xlmr_config).to(device))

In [ ]:
from transformers import Trainer
trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=panx_de_encoded["train"],
                  eval_dataset=panx_de_encoded["validation"],
                  tokenizer=xlmr_tokenizer)

In [ ]:
trainer.train()